**Paddy Doctor**

#Import data and libraries

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import librairies
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Model, optimizers
from tensorflow.keras.applications import xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import to_categorical

!pip install tensorflow-addons
!pip install -q -U keras-tuner
import tensorflow_addons as tfa
import keras_tuner as kt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.2 MB/s 
     |████████████████████████████████| 133 kB 5.0 MB/s 


##data exploration

In [ ]:
df_meta = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/paddy/train.csv")
df_meta.head()

,image_id,label,variety,age
0,100330.jpg,bacterial_leaf_blight,ADT45,45
1,100365.jpg,bacterial_leaf_blight,ADT45,45
2,100382.jpg,bacterial_leaf_blight,ADT45,45
3,100632.jpg,bacterial_leaf_blight,ADT45,45
4,101918.jpg,bacterial_leaf_blight,ADT45,45


In [ ]:
train_path = "/content/drive/MyDrive/Colab Notebooks/paddy/train_images"
test_path = "/content/drive/MyDrive/Colab Notebooks/paddy/test_images"

In [ ]:
disease_list = os.listdir(train_path)
n_classes = len(disease_list)
print(disease_list)
print("Total diseases number:", n_classes)

['brown_spot', 'downy_mildew', 'tungro', 'hispa', 'normal', 'bacterial_leaf_streak', 'blast', 'dead_heart', 'bacterial_panicle_blight', 'bacterial_leaf_blight']
Total diseases number: 10


In [ ]:
# Counts the number of training and testing samples in the directories
training_samples = sum([len(files) for r, d, files in os.walk(train_path)])
testing_samples = sum([len(files) for r, d, files in os.walk(test_path)])

In [ ]:
print(training_samples)
print(testing_samples)

6411
3469


## ResNet152v2

### preprocessing

In [ ]:
img_width = 200
img_height = 200
batch_size = 32

epochs = 100
epochs_ft = 200

nb_classes = 10

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.3,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=False,
    validation_split=0.2,
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input,
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    subset="training",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
)

valid_generator = train_datagen.flow_from_directory(
    train_path,
    subset="validation",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
)

Found 5131 images belonging to 10 classes.
Found 1280 images belonging to 10 classes.


In [ ]:
# images for prediction
test_generator = test_datagen.flow_from_directory(
    directory=test_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    classes=["."],
    shuffle=False,
)

Found 3469 images belonging to 1 classes.


In [ ]:
# f1_score from tensorflow_addons
f1_tfa = tfa.metrics.F1Score(num_classes=n_classes, average="macro")

In [ ]:
def plot_history(dict_history, first_score, second_score):

    """
creates two subplots for plotting 2 different metrics
called first_score and second_score against epoch number,
values obtained from the train and the validation datasets
are visualized
  """

    with plt.style.context("seaborn-whitegrid"):
        fig = plt.figure(figsize=(25, 10))

        # first suplot
        plt.subplot(1, 2, 1)
        plt.plot(dict_history.history[first_score], color="g")
        plt.plot(
            dict_history.history["val_" + first_score], linestyle="--", color="orange"
        )
        plt.title(
            "Pre-trained ResNet152v2 on Paddy dataset " + first_score, fontsize=18
        )
        plt.ylabel(first_score)
        plt.xlabel("epoch")
        plt.legend(["train", "validation"], loc="upper left")

        # second subplot
        plt.subplot(1, 2, 2)
        plt.plot(dict_history.history[second_score], color="g")
        plt.plot(
            dict_history.history["val_" + second_score], linestyle="--", color="orange"
        )
        plt.title(
            "Pre-trained ResNet152v2 on Paddy dataset " + second_score, fontsize=18
        )
        plt.ylabel(second_score)
        plt.xlabel("epoch")
        plt.legend(["train", "validation"], loc="upper left")
        plt.show()

##Model building

In [ ]:
# Adds new top to base model
def add_top(base):
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation="relu")(x)

    predictions = Dense(nb_classes, activation="softmax")(x)  # Output layer

    return Model(inputs=base.input, outputs=predictions)

In [ ]:
# Sets up model for transfer learning
def setup_model(model, base):

    for layer in base.layers:
        layer.trainable = False  # freeze base layers

    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
# Import the ResNet152v2 model to use as the base for our model
resnet_base = tf.keras.applications.ResNet152V2(
    input_shape=(img_width, img_height, 3), include_top=False, weights="imagenet"
)

model = add_top(resnet_base)
setup_model(model, resnet_base)

234561536/234545216 [==============================] - 2s 0us/step


#### Train new top layers

In [ ]:
# callbacks : learning rate scheduler


def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


schedul = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
# callbacks : early stopping
early_stp = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)

In [ ]:
# Train new top layers
model.fit_generator(
    train_generator, epochs=epochs, validation_data=valid_generator, verbose=1,
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/100
161/161 [==============================] - 98s 558ms/step - loss: 2.3944 - accuracy: 0.1495 - val_loss: 2.1151 - val_accuracy: 0.2266
Epoch 2/100
161/161 [==============================] - 85s 531ms/step - loss: 1.9701 - accuracy: 0.2982 - val_loss: 1.8068 - val_accuracy: 0.3633
Epoch 3/100
161/161 [==============================] - 86s 531ms/step - loss: 1.7330 - accuracy: 0.3970 - val_loss: 1.7270 - val_accuracy: 0.4172
Epoch 4/100
161/161 [==============================] - 86s 532ms/step - loss: 1.6219 - accuracy: 0.4407 - val_loss: 1.5964 - val_accuracy: 0.4445
Epoch 5/100
161/161 [==============================] - 85s 530ms/step - loss: 1.5698 - accuracy: 0.4537 - val_loss: 1.5500 - val_accuracy: 0.4859
Epoch 6/100
161/161 [==============================] - 85s 529ms/step - loss: 1.5192 - accuracy: 0.4769 - val_loss: 1.4875 - val_accuracy: 0.4906
Epoch 7/100
161/161 [==============================] - 85s 530ms/step - loss: 1.4398 - accuracy: 0.5020 - val_loss: 1.4251 -

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/paddy/paddy_resnet_top_layers")
print("the model is saved")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/paddy/paddy_resnet_top_layers/assets
the model is saved


In [ ]:
# load model
model = load_model(
    "/content/drive/MyDrive/Colab Notebooks/paddy/paddy_resnet_top_layers"
)

#### Setup model for fine tuning

In [ ]:
# Adam + lookahead
opt = tf.keras.optimizers.Adam(1e-3)
opt = tfa.optimizers.Lookahead(opt)

In [ ]:
# Setup model for fine tuning
def setup_model(model, trainable):
    for layer in model.layers[: (len(model.layers) - trainable)]:
        layer.trainable = False  # freeze model base

    for layer in model.layers[(len(model.layers) - trainable) :]:
        layer.trainable = True  # allow training layers

    model.compile(
        loss="categorical_crossentropy",
        optimizer=opt,  # schedule in callbacks
        metrics=["accuracy"],
    )

In [ ]:
# Setup model to retrain our top layer plus 2 last blocks
setup_model(model, 26)

In [ ]:
# callbacks : early stopping
early_stp = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

In [ ]:
K.clear_session()

In [ ]:
# Fine-tune the model
history4 = model.fit(
    train_generator,
    epochs=epochs_ft,
    validation_data=valid_generator,
    verbose=1,
    callbacks=[early_stp, lrd],
)

Epoch 1/200
161/161 [==============================] - 98s 550ms/step - loss: 0.6937 - accuracy: 0.7652 - val_loss: 1.2412 - val_accuracy: 0.6164 - lr: 0.0010
Epoch 2/200
161/161 [==============================] - 85s 528ms/step - loss: 0.5522 - accuracy: 0.8189 - val_loss: 0.9116 - val_accuracy: 0.7164 - lr: 0.0010
Epoch 3/200
161/161 [==============================] - 85s 527ms/step - loss: 0.4974 - accuracy: 0.8384 - val_loss: 0.7587 - val_accuracy: 0.7484 - lr: 0.0010
Epoch 4/200
161/161 [==============================] - 85s 530ms/step - loss: 0.4420 - accuracy: 0.8523 - val_loss: 0.6727 - val_accuracy: 0.8039 - lr: 0.0010
Epoch 5/200
161/161 [==============================] - 85s 528ms/step - loss: 0.3924 - accuracy: 0.8694 - val_loss: 0.6800 - val_accuracy: 0.7992 - lr: 0.0010
Epoch 6/200
161/161 [==============================] - ETA: 0s - loss: 0.3723 - accuracy: 0.8788
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.000800000037997961.
161/161 [========================

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

lrd = ReduceLROnPlateau(
    monitor="val_loss", patience=2, verbose=1, factor=0.8, min_lr=1e-6
)

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/paddy/paddy_resnet_fine_tuned_adam")
print("the model is saved")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/paddy/paddy_resnet_fine_tuned_adam/assets
the model is saved


In [ ]:
model = load_model(
    "/content/drive/MyDrive/Colab Notebooks/paddy/paddy_resnet_fine_tuned_adam"
)

In [ ]:
# predict sur test set avec resnet_26 layers
disease_pred = model.predict(test_generator, batch_size=32,)

In [ ]:
# formatting Kaggle submission
predicted_class_indices = np.argmax(disease_pred, axis=1)
print(set(predicted_class_indices))

inv_map = {v: k for k, v in train_generator.class_indices.items()}

predictions = [inv_map[k] for k in predicted_class_indices]

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [ ]:
filenames = test_generator.filenames

results = pd.DataFrame({"image_id": filenames, "label": predictions})
results.image_id = results.image_id.str.replace("./", "")
results.to_csv("/content/drive/MyDrive/Colab Notebooks/submission2.csv", index=False)
results.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


,image_id,label
0,200001.jpg,hispa
1,200002.jpg,normal
2,200003.jpg,blast
3,200004.jpg,blast
4,200005.jpg,blast


###with lr scheduler

In [ ]:
# Fine-tune the model
history4 = model.fit(
    train_generator,
    epochs=epochs_ft,
    validation_data=valid_generator,
    verbose=1,
    callbacks=[early_stp, schedul],
)

Epoch 1/200
161/161 [==============================] - 99s 501ms/step - loss: 1.2660 - accuracy: 0.5794
Epoch 2/200
161/161 [==============================] - 81s 500ms/step - loss: 0.8178 - accuracy: 0.7252
Epoch 3/200
161/161 [==============================] - 80s 493ms/step - loss: 0.7119 - accuracy: 0.7652
Epoch 4/200
161/161 [==============================] - 80s 499ms/step - loss: 0.6287 - accuracy: 0.7872
Epoch 5/200
161/161 [==============================] - 80s 493ms/step - loss: 0.5926 - accuracy: 0.8014
Epoch 6/200
161/161 [==============================] - 81s 500ms/step - loss: 0.5698 - accuracy: 0.8162
Epoch 7/200
161/161 [==============================] - 80s 496ms/step - loss: 0.5418 - accuracy: 0.8258
Epoch 8/200
161/161 [==============================] - 79s 493ms/step - loss: 0.4896 - accuracy: 0.8406
Epoch 9/200
161/161 [==============================] - 81s 500ms/step - loss: 0.4540 - accuracy: 0.8480
Epoch 10/200
161/161 [==============================] - 80s 495m

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/paddy/paddy_resnet_fine_tuned_26")
print("the model is saved")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/paddy/paddy_resnet_fine_tuned_26/assets
the model is saved


In [ ]:
# predict sur test set avec resnet_26 layers
disease_pred = model.predict(test_generator, batch_size=32,)

In [ ]:
# formatting Kaggle submission
predicted_class_indices = np.argmax(disease_pred, axis=1)
print(set(predicted_class_indices))

inv_map = {v: k for k, v in train_generator.class_indices.items()}

predictions = [inv_map[k] for k in predicted_class_indices]

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [ ]:
filenames = test_generator.filenames

results = pd.DataFrame({"image_id": filenames, "label": predictions})
results.image_id = results.image_id.str.replace("./", "")
results.to_csv("submission.csv", index=False)
results.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


,image_id,label
0,200001.jpg,hispa
1,200002.jpg,normal
2,200003.jpg,blast
3,200004.jpg,blast
4,200005.jpg,blast
